In [1]:
import pandas as pd
import numpy as np
import copy

# ArcGIS packages, comment out everything below if you don't have arcpy installed
import arcpy
from arcpy.sa import *

ArcPy Environment

In [2]:
# Base directory
arcpy.env.workspace = r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023"

# Allows for file overwrite (necessary for running code multiple times)
arcpy.env.overwriteOutput = True

Pregenerated Files

In [3]:
# Flow Direction

# After it rains, water will flow downhill due to gravity. 
# To model where the water ends up, we must first find the altitudes of each individual raster.
# Then, we can run a "gradient descent" through ArcGis's built-in Flow Direction Tool.

# Relevant Sources:
# https://www.gpsvisualizer.com/elevation

# 1. Turn WSS California Spatial Layer into Raster Layer w/ Polygon to Raster (Cellsize = 0.016)
# 2. Turn resulting raster into point layer w/ Raster to Point
# 3. Generate coordinates w/ Add X,Y Coordinates (Output Coordinate System = WGS 1984)
# 4. Export attribute table from resulting point layer w/ Table to Excel
# Outside of ArcGIS
# 5. Create a copy of the resulting Excel file
# 6. Manually change labels from Point_Y, Point_X to Latitude, Longitude respectively. Delete all other rows except ObjectID.
# 7. Go to https://www.gpsvisualizer.com/elevation, upload edited Excel file into Solution 1, DEM database
# 8. Change the resulting output to text file and make sure outputs are as expected w/ Google visualizer
# 9. Open the text file with Pandas and export as an excel file.
# 10. Copy the altitude column of the resulting excel file into the original dataset.
# 11. Click dropdown menu in Add Data and choose X,Y Data
# 12. Add the Excel file, setting z-field as altitude
# 13. Turn resulting point layer to a raster w/ Point to Raster (Cellsize = 0.016)
# 14. Create flow direction raster w/ Flow Direction (Type = MDF)

flowdirection_path = r"arcgis file\flow direction.crf"

""""""

# Point Data

# Although we have classified each square mile of soil into a soil type and an altitude, we can go even further and find the county label and evapotranspiration.

# Relevant Sources
# https://hub.arcgis.com/datasets/CALFIRE-Forestry::california-counties/explore?location=37.177658%2C-119.270300%2C6.84
# https://gis.water.ca.gov/arcgis/rest/services/Climatology/i04_CIMIS_Reference_Evapotranspiration_Zones_v1999/FeatureServer

# 1. Extract the California County shapefile from https://hub.arcgis.com/datasets/CALFIRE-Forestry::california-counties/explore?location=37.177658%2C-119.270300%2C6.84 
# 2. Extract the California Evapotranspiration shapefile from https://gis.water.ca.gov/arcgis/rest/services/Climatology/i04_CIMIS_Reference_Evapotranspiration_Zones_v1999/FeatureServer
# 3. Add both shapefiles to the ArcGis Project and run Spatial Join by Closest between them and the point layer with elevation (note: this may take a long time, like 5 minutes per spatial join.)
# 4. Export the resulting attribute table.

datapath = r"arcgis setup\point layer final.xlsx"

# Physical Soil Properties
# Maximal depth assumed to be 60 in

# Relevant Sources
# https://websoilsurvey.nrcs.usda.gov/app/WebSoilSurvey.aspx

# 1. Go to https://websoilsurvey.nrcs.usda.gov/app/WebSoilSurvey.aspx and download California STATSGO2
# 2. Open the Access Database and follow directions in readme file
# 3. Generate soil report for Physical Soil Properties with all Map Unit Symbols
# 4. Export soil report as text file

# s8369 is not inclduded (water)
propertiespath = r"arcgis setup\physical soil properties.txt"
permeabilitypath = r"arcgis setup\soil permeability.xlsx"
storagepath = r"arcgis setup\soil storage.xlsx"

""""""

# Weight Layer

# We need an empty excel file we can add data to that can later be turned into a weight point layer

# Relevant Sources
# None

# 1. Make a copy of the point layer excel file and remove everything except ID, Latitude, and Longitude

emptypath = r"arcgis setup\weight layer.xlsx"

PSP Text File Manipulation

In [58]:
f = open(propertiespath, 'r')

# Setting current map unit and soil name to none
currmu = ""
currsoil = ""

# Creating empty sets for overall dataset
propertyall = {}
propertymu = {"":""}

# Creating a filler variable
fill = []
c = 0

# Reading through dataset
for line in f.readlines():

    # Splitting into lines
    line = line.split()
    
    # If we actually care about the information there
    if len(line) == 1 or len(line) == 15 or len(line) == 11:
        
        # If it displays a map unit
        if len(line) == 1:
            
            # If it's not the old map unit
            if line[0] != currmu:
                
                del propertymu[""]
                propertymu[currsoil] = fill
                propertyall[currmu] = propertymu

                currsoil = ""
                propertymu = {"":""}
                currmu = line[0]
        
        # If it displays a soil name
        elif len(line) == 15:
            
            # If it's really a new soil
            if line[0] + str(c) != currsoil:
                
                c += 1
                propertymu[currsoil] = fill
                
                # Adding a counter in case duplicate soil names
                currsoil = line[0] + str(c)
                
                fill = []
            
            # Add in depth, permeability, and capacity
            fill.append([line[1], line[6], line[7]])

        else:
            
            # Add in depth, permeability, and capacity
            fill.append([line[0], line[5], line[6]])

del propertyall[""]
f.close()

In [65]:
stor = {}
perm = {}

# Conversion factors:
s = 5.33 * 10 ** (-5) # in per in to maf/mi per in
p = 0.0864 # micro m/sec to in/day through Darcy's Law

# Going through map units
for key in propertyall.keys():
    
    # Creating initial values, setting maximal depth to 120
    initstor = np.zeros(120)
    initperm = np.zeros(120)
    
    # Looping through soil names
    for val in propertyall[key].keys():
        
        # Looping through depths
        for layer in propertyall[key][val]:
            
            # If the depth is non-empty and the values exist
            if layer[0] != "" and len(layer[2].split("-")) == 2 and len(layer[1].split("-")) == 2:

                depth = layer[0].split("-")
                awc = layer[2].split("-")
                speeds = layer[1].split("-")
                                
                # If the secondary values are empty, replace them with the first ones
                if speeds[1] == "":
                    speeds[1] = speeds[0] 
                if awc[1] == "":
                    awc[1] = awc[0]
                               
                # If the depth is larger than the max
                if int(depth[0]) >= 120:
                    break
                
                # Adding water capacity and permeability per depth range to initial values
                initstor[int(depth[0]):min(int(depth[1]), 120)] += (float(awc[0])/2 + float(awc[1])/2) * s
                initperm[int(depth[0]):min(int(depth[1]), 120)] += (float(speeds[0])/2 + float(speeds[1])/2) * p
                    
    # Averaging across soils in a single map unit
    stor[key] = initstor/len(propertyall[key])
    perm[key] = initperm/len(propertyall[key])

# Turning them into excel files so this is a one and done occurance:
col = [i+1 for i in range(120)]
col.insert(0, "MUSYM")

stordf = pd.DataFrame(columns=col)
permdf = pd.DataFrame(columns=col)

for key in propertyall.keys():
    stordf.loc[len(stordf)] = [key.strip(":")] + list(stor[key])
    permdf.loc[len(permdf)] = [key.strip(":")] + list(perm[key])
    
stordf.to_excel(r"arcgis setup\soil storage.xlsx")
permdf.to_excel(r"arcgis setup\soil permeability.xlsx")

Opening Files

In [4]:
data = pd.read_excel(datapath)
empty = pd.read_excel(emptypath)

permeability = pd.read_excel(permeabilitypath).iloc[:, 1:]
storage = pd.read_excel(storagepath).iloc[:, 1:]

In [5]:
data

,ID,MUSYM,Latitude,Longitude,Altitude,NAME,Zone,January_Monthly_Avg_ETo,February_Monthly_Avg_ETo,March_Monthly_Avg_ETo,April_Monthly_Avg_ETo,May_Monthly_Avg_ETo,June_Monthly_Avg_ETo,July_Monthly_Avg_ETo,August_Monthly_Avg_ETo,September_Monthly_Avg_ETo,October_Monthly_Avg_ETo,November_Monthly_Avg_ETo,December_Monthly_Avg_ETo
0,1,s509,41.998411,-123.633842,692.9,Del Norte,3,1.86,2.24,3.72,4.8,5.27,5.7,5.58,5.27,4.2,3.41,2.4,1.86
1,2,s509,41.998411,-123.617842,580.5,Del Norte,3,1.86,2.24,3.72,4.8,5.27,5.7,5.58,5.27,4.2,3.41,2.4,1.86
2,3,s509,41.998411,-123.601842,753.1,Del Norte,13,1.24,1.96,3.10,4.8,6.51,7.8,8.99,7.75,5.7,3.72,1.8,0.93
3,4,s512,41.998411,-123.585842,901.9,Del Norte,13,1.24,1.96,3.10,4.8,6.51,7.8,8.99,7.75,5.7,3.72,1.8,0.93
4,5,s512,41.998411,-123.569842,1244.6,Del Norte,13,1.24,1.96,3.10,4.8,6.51,7.8,8.99,7.75,5.7,3.72,1.8,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162519,162520,s1002,32.542411,-117.105842,24.0,San Diego,1,0.93,1.40,2.48,3.3,4.03,4.5,4.65,4.03,3.3,2.48,1.2,0.62
162520,162521,s998,32.542411,-117.089842,93.1,San Diego,1,0.93,1.40,2.48,3.3,4.03,4.5,4.65,4.03,3.3,2.48,1.2,0.62
162521,162522,s998,32.542411,-117.073842,60.8,San Diego,1,0.93,1.40,2.48,3.3,4.03,4.5,4.65,4.03,3.3,2.48,1.2,0.62
162522,162523,s1002,32.542411,-117.057842,11.5,San Diego,3,1.86,2.24,3.72,4.8,5.27,5.7,5.58,5.27,4.2,3.41,2.4,1.86


In [6]:
permeability

,MUSYM,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,s275,1.027080,1.027080,1.027080,1.027080,1.027080,1.027080,1.027080,0.967032,0.967032,...,0,0,0,0,0,0,0,0,0,0
1,s295,0.792288,0.792288,0.792288,0.792288,0.792288,0.792288,0.792288,0.792288,0.792288,...,0,0,0,0,0,0,0,0,0,0
2,s503,1.563192,1.563192,1.563192,1.563192,1.532705,1.463030,1.463030,1.463030,1.463030,...,0,0,0,0,0,0,0,0,0,0
3,s504,1.420330,1.420330,1.420330,1.420330,1.377648,1.280102,1.280102,1.280102,1.280102,...,0,0,0,0,0,0,0,0,0,0
4,s505,1.111104,1.111104,1.043579,0.959173,0.959173,0.959173,0.959173,0.959173,0.655311,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,s6360,0.970128,0.970128,0.970128,0.970128,0.970128,0.970128,0.970128,0.970128,0.909168,...,0,0,0,0,0,0,0,0,0,0
700,s6361,1.296756,1.296756,1.296756,1.296756,1.296756,1.296756,1.296756,1.296756,1.296756,...,0,0,0,0,0,0,0,0,0,0
701,s6362,1.031678,1.031678,1.031678,1.031678,1.031678,1.031678,1.031678,1.031678,1.031678,...,0,0,0,0,0,0,0,0,0,0
702,s6363,0.971215,0.971215,0.971215,0.971215,0.971215,0.971215,0.749481,0.749481,0.554963,...,0,0,0,0,0,0,0,0,0,0


Relevant Functions

In [5]:
# Because our time scale for our rainfall model is in months, we would like a way to distribute the rain across multiple days.

def randomsplit(x, n):
    
    # Create a random number generator environment
    rng = np.random.default_rng()

    # Create a set of n-1 "seperators" between 0 and 1
    arr = list(rng.random((n-1,)))
    arr.sort()
    
    arr.insert(0, 0)
    arr.append(1)

    # Enacts a sort of prefix difference
    splitarr = np.zeros(n)
    for i in range(len(arr)):
        if i >= 1:
            splitarr[i-1] = arr[i] - arr[i-1]

    return(splitarr*x)

# We would also like a way for our model to generate a month from an index, but remainder starts at 0, not 12

def month(x):
    
    monthlist = [1,2,3,4,5,6,7,8,9,10,11,0]
    
    return(monthlist.index(x % 12))

Water Location Model

In [11]:
# Rainfall Model generated values
tempmaxlist = np.asarray([285.99548533, 290.60936567, 296.47407367, 298.49854467, 303.44844967, 306.181745  , 307.52888167, 307.76914533, 308.046397  , 301.50453667, 289.79283067, 288.3251])-273.15
tempminlist = np.asarray([279.52412867, 282.03942567, 282.50230633, 283.03047767, 286.44331733, 289.196742  , 289.501551  , 289.73098367, 288.75478433, 285.118149  , 281.11255067, 283.51997433])-273.15
rainlist = [3.73584774e-01, 2.92836556e+00, 7.29598878e+00, 6.70503856e+00, 7.42052972e-01, 2.71029869e+00, 6.95292694e-01, 0.00000000e+00, 1.17418608e-02, 8.83337616e-06, 1.29752368e-03, 6.23148740e-02,]

tempavglist = (tempmaxlist + tempminlist)/2
tempdiflist = (tempmaxlist - tempminlist)

tempavgmax = max(tempavglist)
tempavgmin = min(tempavglist)

# Conversion factor from kg/m^2 to maf
c = 2.0997337 * 10 ** -6
rainlist = c * np.asarray(rainlist)

In [6]:
# Creating states

# How much water it is currently holding
holdcur = {}

# How much water it can hold
holdmax = {}

# How fast water flows through
holdflow = {}

# Set empty arrays for values
for i in data.values:

    # Index
    index = i[0]

    # Map Unit Symbol
    musym = i[1]

    if musym != "s8369":
        
        # Returns a list of all values for a given soil
        permlist = np.asarray(permeability[permeability["MUSYM"] == musym].iloc[0, 1:], dtype=float)
        storlist = np.asarray(storage[storage["MUSYM"] == musym].iloc[0, 1:], dtype=float)

        permlist = list(np.ndarray.round(permlist))
        
        if (0 in permlist):
            zeroloc = permlist.index(0)
            permlist = permlist[:zeroloc]
            storlist = storlist[:zeroloc]
            
        # Average permeability rate
        avgperm = np.round(np.mean(permlist))

        if np.isnan(avgperm) or avgperm == 0:
            
            # If the ground can't hold any water, have it act as a water reservoir
            holdcur[index] = [0]
            holdmax[index] = [np.inf]
            holdflow[index] = [0]
            
        else:
        
            # The number of states
            numstate = int(np.ceil(len(storlist)/avgperm))

            # If the ground can hold water, create a 0 value for each "state" 
            holdcur[index] = np.zeros(numstate)

            # To set the max amount of water for any given state, we need to sum values from storage
            maxlist = []

            for j in range(numstate):
                maxlist.append(sum(storlist[j * int(avgperm) : min((j+1) * int(avgperm), len(storlist))]))
                
            holdmax[index] = maxlist
            
            holdflow[index] = avgperm
    
    else:
        
        holdcur[index] = [0]
        holdmax[index] = [np.inf]
        holdflow[index] = 0

c:\Users\thewa\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\thewa\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
convert = 2.0997337 * 10 ** -6

# Empty list of the amount of groundwater avaliable
availdict = {}

# Run through a year
for monthnum in range(12):
    
    tempcur = copy.deepcopy(holdcur)
    tempmax = copy.deepcopy(holdmax)
    tempflow = copy.deepcopy(holdflow)
    
    # For each month, the amount of groundwater avaliable
    availlist = []
    remlist = np.zeros(len(data))
    
    # Split rain into 5 weeks and run through each part
    for x in randomsplit(rainlist[monthnum], 5): 
        c = 0
        avail = 0
        # The amount of water on the surface

        # Runs through all the points
        for row in data.values:
            
            # How much water the ground holds at every state
            old = tempcur[row[0]]
            
            # Maximal amount of water the ground can hold at every state
            maximum = tempmax[row[0]]
            
            # The amount of water lost to evapotranspiration
            etovar = row[7] * convert
            
            # Water evaporated from surface
            tempcur[row[0]][0] -= min(etovar, old[0])
            
            # If water can go through
            if tempflow[row[0]] != 0:
                
                newlist = copy.deepcopy(old)
                for i in range(1, len(old)):
                    a = newlist[-i]
                    b = newlist[-i-1]

                    newlist[-i] = min(maximum[-i], a + b)
                    newlist[-i-1] = max(0, a + b - maximum[-i])
                
                new = min(x + newlist[0], maximum[0])
                rem = max(0, newlist[0] + x - maximum[0])

                newlist[0] = new
                tempcur[row[0]] = newlist
                
            else:
                
                tempcur[row[0]] = [old[0] + x]
                rem = old[0] + x
    
            remlist[c] += rem            
            c += 1
            
    avail = 0
    for row in data.values:
        # The amount of water inside the ground (total - surface)
        avail += sum(tempcur[row[0]]) - tempcur[row[0]][0]
    availdict[monthnum] = avail
    """
    empty["Rem"] = remlist
    print(sum(remlist))
    empty.to_excel(r"arcgis setup\surface layer.xlsx")
    
    # Creates initial point file w/ coordinates WGS 1984
    projection = arcpy.SpatialReference(4326)
    
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer"
    arcpy.env.overwriteOutput = True
    surfacelayer = arcpy.management.XYTableToPoint(r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\surface layer.xlsx/Sheet1$", r"surface.shp", "longitude", "latitude", coordinate_system = projection)

    # Turns it into a raster file
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Weight Layer"
    arcpy.env.overwriteOutput = True
    weightlayer = arcpy.conversion.PointToRaster(r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer\surface.shp", "Rem", r"weight_" + str(monthnum) + ".tif", "MEAN", "", 0.016)
    
    # Runs flow accumulation with rastered as weight
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Flow Accumulation Raster Layer"
    arcpy.env.overwriteOutput = True
    flow = FlowAccumulation(flowdirection_path, weightlayer, flow_direction_type = "MFD")
    
    # Saves it to check if needed
    flow.save(r"raster surface.tif")
    
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Flow Accumulation Point Layer"
    arcpy.env.overwriteOutput = True
    
    # Turns raster into point
    point = arcpy.conversion.RasterToPoint(flow, r"point surface.shp")

    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Final Results"
    arcpy.env.overwriteOutput = True

    # Saves it and reads it
    loc = r"Month " + str(monthnum) + ".xlsx"
    water = arcpy.conversion.TableToExcel(point, loc)
    
    del water
    del point
    del flow
    
    del weightlayer
    del surfacelayer
    """

5 Year Ahead

In [16]:
# Rainfall Model generated values
tempmaxlist = np.asarray([306.87821367, 306.002554  , 295.263266  , 289.73215667, 285.90098197, 286.32827292, 288.24651583, 290.56602751, 300.37107491, 304.12201333, 307.35666512, 306.99192124])-273.15
tempminlist = np.asarray([277.73523167, 276.925101  , 277.45411667, 279.968295  , 282.08340433, 285.43810933, 288.59289033, 289.48194267, 288.928506  , 287.964809  , 282.58445533, 278.86252367])-273.15
rainlist = [8.51333697e-01, 2.74756270e+00, 7.00556261e+00, 5.43645570e+00, 3.42437672e+00, 1.03481404e+00, 2.20333550e+00, 7.11763005e-01, 9.48914504e-03, 8.15700616e-03, 3.88041491e-02, 4.04297489e-01]

tempavglist = (tempmaxlist + tempminlist)/2
tempdiflist = (tempmaxlist - tempminlist)

tempavgmax = max(tempavglist)
tempavgmin = min(tempavglist)


# Conversion factor from kg/m^2 to maf
c = 2.0997337 * 10 ** -6
rainlist = c * np.asarray(rainlist)

convert = 2.0997337 * 10 ** -6

# Empty list of the amount of groundwater avaliable
availdict5 = {}

# Run through a year
for monthnum in range(12):
    
    tempcur = copy.deepcopy(holdcur)
    tempmax = copy.deepcopy(holdmax)
    tempflow = copy.deepcopy(holdflow)
    
    # For each month, the amount of groundwater avaliable
    remlist = np.zeros(len(data))
    
    # Split rain into 5 weeks and run through each part
    for x in randomsplit(rainlist[monthnum], 5): 
        c = 0
        avail = 0
        # The amount of water on the surface

        # Runs through all the points
        for row in data.values:
            
            # How much water the ground holds at every state
            old = tempcur[row[0]]
            
            # Maximal amount of water the ground can hold at every state
            maximum = tempmax[row[0]]
            
            # The amount of water lost to evapotranspiration
            etovar = row[7] * convert
            
            # Water evaporated from surface
            tempcur[row[0]][0] -= min(etovar, old[0])
            
            # If water can go through
            if tempflow[row[0]] != 0:
                
                newlist = copy.deepcopy(old)
                for i in range(1, len(old)):
                    a = newlist[-i]
                    b = newlist[-i-1]

                    newlist[-i] = min(maximum[-i], a + b)
                    newlist[-i-1] = max(0, a + b - maximum[-i])
                
                new = min(x + newlist[0], maximum[0])
                rem = max(0, newlist[0] + x - maximum[0])

                newlist[0] = new
                tempcur[row[0]] = newlist
                
            else:
                
                tempcur[row[0]] = [old[0] + x]
                rem = old[0] + x
    
            remlist[c] += rem            
            c += 1
            
    avail = 0
    for row in data.values:
        # The amount of water inside the ground (total - surface)
        avail += sum(tempcur[row[0]]) - tempcur[row[0]][0]
    availdict5[monthnum] = avail
    """
    empty["Rem"] = remlist
    print(sum(remlist))
    empty.to_excel(r"arcgis setup\surface layer.xlsx")
    
    # Creates initial point file w/ coordinates WGS 1984
    projection = arcpy.SpatialReference(4326)
    
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer"
    arcpy.env.overwriteOutput = True
    surfacelayer = arcpy.management.XYTableToPoint(r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\surface layer.xlsx/Sheet1$", r"surface.shp", "longitude", "latitude", coordinate_system = projection)

    # Turns it into a raster file
    arcpy.env.workspace = r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Weight Layer"
    arcpy.env.overwriteOutput = True
    weightlayer = arcpy.conversion.PointToRaster(r"C:\\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer\surface.shp", "Rem", r"weight5_" + str(monthnum) + ".tif", "MEAN", "", 0.016)

    del weightlayer
    del surfacelayer
    """

10 Year Ahead

In [8]:
# Rainfall Model generated values
tempmaxlist = np.asarray([306.60847792, 303.53192441, 298.85749966, 293.78375404, 289.56020822, 287.23020042, 287.42055942, 290.15619713, 294.77650699, 300.06900072, 304.58020088, 307.09150173])-273.15
tempminlist = np.asarray([277.69608673, 276.43842493, 276.52887765, 278.00521733, 280.52492212, 283.3828243 , 285.75260116, 287.00854395, 286.84783015, 285.37842589, 283.005523  , 280.32173414])-273.15
rainlist = [5.17108107e+00, 4.25557307e+00, 5.90597891e-01, 8.47976414e-01, 9.42257523e-01, 3.45543759e-01, 0.00000000e+00, 2.12546396e-01, 7.95816553e-02, 2.34285068e-01, 1.80429764e+00, 3.19692566e+00]

tempavglist = (tempmaxlist + tempminlist)/2
tempdiflist = (tempmaxlist - tempminlist)

tempavgmax = max(tempavglist)
tempavgmin = min(tempavglist)

# Conversion factor from kg/m^2 to maf
c = 2.0997337 * 10 ** -6
rainlist = c * np.asarray(rainlist)

convert = 2.0997337 * 10 ** -6

# Empty list of the amount of groundwater avaliable
availdict10 = {}

# Run through a year
for monthnum in range(12):
    
    tempcur = copy.deepcopy(holdcur)
    tempmax = copy.deepcopy(holdmax)
    tempflow = copy.deepcopy(holdflow)
    
    # For each month, the amount of groundwater avaliable
    remlist = np.zeros(len(data))
    
    # Split rain into 5 weeks and run through each part
    for x in randomsplit(rainlist[monthnum], 5): 
        c = 0
        avail = 0
        # The amount of water on the surface

        # Runs through all the points
        for row in data.values:
            
            # How much water the ground holds at every state
            old = tempcur[row[0]]
            
            # Maximal amount of water the ground can hold at every state
            maximum = tempmax[row[0]]
            
            # The amount of water lost to evapotranspiration
            etovar = row[7] * convert
            
            # Water evaporated from surface
            tempcur[row[0]][0] -= min(etovar, old[0])
            
            # If water can go through
            if tempflow[row[0]] != 0:
                
                newlist = copy.deepcopy(old)
                for i in range(1, len(old)):
                    a = newlist[-i]
                    b = newlist[-i-1]

                    newlist[-i] = min(maximum[-i], a + b)
                    newlist[-i-1] = max(0, a + b - maximum[-i])
                
                new = min(x + newlist[0], maximum[0])
                rem = max(0, newlist[0] + x - maximum[0])

                newlist[0] = new
                tempcur[row[0]] = newlist
                
            else:
                
                tempcur[row[0]] = [old[0] + x]
                rem = old[0] + x
    
            remlist[c] += rem            
            c += 1
            
    avail = 0
    for row in data.values:
        # The amount of water inside the ground (total - surface)
        avail += sum(tempcur[row[0]]) - tempcur[row[0]][0]
    availdict10[monthnum] = avail
    
    empty["Rem"] = remlist
    print(sum(remlist))
    empty.to_excel(r"arcgis setup\surface layer.xlsx")
    
    # Creates initial point file w/ coordinates WGS 1984
    projection = arcpy.SpatialReference(4326)
    
    arcpy.env.workspace = r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer"
    arcpy.env.overwriteOutput = True
    surfacelayer = arcpy.management.XYTableToPoint(r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\surface layer.xlsx/Sheet1$", r"surface.shp", "longitude", "latitude", coordinate_system = projection)

    # Turns it into a raster file
    arcpy.env.workspace = r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Weight Layer"
    arcpy.env.overwriteOutput = True
    weightlayer = arcpy.conversion.PointToRaster(r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer\surface.shp", "Rem", r"weight10_" + str(monthnum) + ".tif", "MEAN", "", 0.016)

    del weightlayer
    del surfacelayer

0.03284399624852561
0.08236873767243676
0.001856427147437738
0.0026654454059616603
0.0029617993430570565
0.0010861481637686903
0.0
0.0006680973732506425
0.0002501491244521835
0.0007364285702715496
0.005876914228799249
0.010377251710707785


15 Years Ahead

In [9]:
# Rainfall Model generated values
tempmaxlist = np.asarray([292.44265061, 288.8100255 , 287.43383565, 288.67891113, 292.23063894, 297.16795176, 302.19940633, 306.00451796, 307.58705039, 306.5394447 , 303.14592164, 298.30065618])-273.15
tempminlist = np.asarray([279.44911504, 277.39945294, 276.49427878, 276.97395522, 278.71883304, 281.27513718, 283.97203111, 286.10046976, 287.10335104, 286.7224697 , 285.06293719, 282.5611591])-273.15
rainlist = [1.92036063e+00, 4.47482520e+00, 2.48078120e+00, 1.12281846e+00, 2.59069966e+00, 7.50999641e-01, 0.00000000e+00, 1.32708325e-01, 4.34867983e-02, 2.86640823e-01, 2.01503957e-01, 1.05283881e+00]

tempavglist = (tempmaxlist + tempminlist)/2
tempdiflist = (tempmaxlist - tempminlist)

tempavgmax = max(tempavglist)
tempavgmin = min(tempavglist)

# Conversion factor from kg/m^2 to maf
c = 2.0997337 * 10 ** -6
rainlist = c * np.asarray(rainlist)

convert = 2.0997337 * 10 ** -6

# Empty list of the amount of groundwater avaliable
availdict15 = {}

# Run through a year
for monthnum in range(12):
    
    tempcur = copy.deepcopy(holdcur)
    tempmax = copy.deepcopy(holdmax)
    tempflow = copy.deepcopy(holdflow)
    
    # For each month, the amount of groundwater avaliable
    remlist = np.zeros(len(data))
    
    # Split rain into 5 weeks and run through each part
    for x in randomsplit(rainlist[monthnum], 5): 
        c = 0
        avail = 0
        # The amount of water on the surface

        # Runs through all the points
        for row in data.values:
            
            # How much water the ground holds at every state
            old = tempcur[row[0]]
            
            # Maximal amount of water the ground can hold at every state
            maximum = tempmax[row[0]]
            
            # The amount of water lost to evapotranspiration
            etovar = row[7] * convert
            
            # Water evaporated from surface
            tempcur[row[0]][0] -= min(etovar, old[0])
            
            # If water can go through
            if tempflow[row[0]] != 0:
                
                newlist = copy.deepcopy(old)
                for i in range(1, len(old)):
                    a = newlist[-i]
                    b = newlist[-i-1]

                    newlist[-i] = min(maximum[-i], a + b)
                    newlist[-i-1] = max(0, a + b - maximum[-i])
                
                new = min(x + newlist[0], maximum[0])
                rem = max(0, newlist[0] + x - maximum[0])

                newlist[0] = new
                tempcur[row[0]] = newlist
                
            else:
                
                tempcur[row[0]] = [old[0] + x]
                rem = old[0] + x
    
            remlist[c] += rem            
            c += 1
            
    avail = 0
    for row in data.values:
        # The amount of water inside the ground (total - surface)
        avail += sum(tempcur[row[0]]) - tempcur[row[0]][0]
    availdict15[monthnum] = avail
    
    empty["Rem"] = remlist
    print(sum(remlist))
    empty.to_excel(r"arcgis setup\surface layer.xlsx")
    
    # Creates initial point file w/ coordinates WGS 1984
    projection = arcpy.SpatialReference(4326)
    
    arcpy.env.workspace = r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer"
    arcpy.env.overwriteOutput = True
    surfacelayer = arcpy.management.XYTableToPoint(r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\surface layer.xlsx/Sheet1$", r"surface.shp", "longitude", "latitude", coordinate_system = projection)

    # Turns it into a raster file
    arcpy.env.workspace = r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Weight Layer"
    arcpy.env.overwriteOutput = True
    weightlayer = arcpy.conversion.PointToRaster(r"C:\Users\thewa\OneDrive\Documents\MTF Finals 2023\arcgis setup\Surface Layer\surface.shp", "Rem", r"weight15_" + str(monthnum) + ".tif", "MEAN", "", 0.016)

    del weightlayer
    del surfacelayer

0.006065544760880658
0.01747528709294126
0.007809910315371382
0.003529356779887791
0.008161682509917348
0.0023606181845787226
0.0
0.00041714225698276613
0.00013669211175573387
0.0009009984855857259
0.0006333876598467947
0.0033093896516473817


Analysis

In [18]:
xls = pd.ExcelFile(r"arcgis setup\Final Results\final all.xlsx")

In [19]:
adjlevel = []

for month in range(12):
    df = pd.read_excel(xls, 'RasterT_FlowAcc' + str(month+1))
    adjlevel.append(sum((df["grid_code"] - np.maximum(np.zeros(len(df["grid_code"])), np.minimum(df["grid_code"], (700*(tempavglist[month])/(100 - data["Altitude"]) + 15 * (0.0023 * data["Altitude"] + 0.37 * tempavglist[month] + 0.53 * tempdiflist[month] + 0.35 * (tempavgmax - tempavgmin) - 10.9))/(80-tempavglist[month]) * c)))))
    print(month)

print(adjlevel)

0
1


KeyboardInterrupt: 

In [ ]:
x = []
# High pred (pessimistic)
for month in range(12): # weekly/5 * 5/7 - daily loss -> daily total
    x.append(adjlevel[month]*5/7 - 7379207/1233481.375475)
print(x)
# Median 
x = []
for month in range(12): # weekly/5 * 5/7 - daily loss -> daily total
    x.append(adjlevel[month]*5/7 - 1471342/1233481.375475)
print(x)
x = []
# Low pred (optimistic)
for month in range(12): # weekly/5 * 5/7 - daily loss -> daily total
    x.append(adjlevel[month]*5/7 - 763884/1233481.375475)
print(x)

[-5.349001562727157, -5.3620025149092685, -5.3915190327572855, -5.415307845721538, -5.413190803628093, -5.430875941062707, -5.414262836124028, -5.406565191073471, -5.396653412693176, -5.328403323911602, -5.317232125127294, -5.308061502537461]
[-0.5594156658789398, -0.5724166180610515, -0.6019331359090683, -0.6257219488733208, -0.6236049067798758, -0.6412900442144902, -0.6246769392758108, -0.6169792942252543, -0.607067515844959, -0.5388174270633852, -0.527646228279077, -0.5184756056892442]
[0.014130083628275836, 0.001129131446164111, -0.028387386401852677, -0.05217619936610518, -0.05005915727266019, -0.06774429470727461, -0.05113118976859521, -0.04343354471803873, -0.0335217663377434, 0.034728322443830395, 0.04589952122813856, 0.05507014381797137]


In [17]:
print(np.asarray(list(availdict.values())) * 1/5 - (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict.values())) * 1/5 - (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict.values())) * 1/5 - (-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 12)

print(np.asarray(list(availdict5.values())) * 1/5 - (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict5.values())) * 1/5 - (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict5.values())) * 1/5 - (-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 12)

print(np.asarray(list(availdict10.values())) * 1/5 - (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict10.values())) * 1/5 - (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict10.values())) * 1/5 - (-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 12)

print(np.asarray(list(availdict15.values())) * 1/5 - (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict15.values())) * 1/5 - (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 12)
print(np.asarray(list(availdict15.values())) * 1/5 - (-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 12)

[-0.00573284  0.0385662  -0.00573284 -0.00573284 -0.00358285 -0.00573284
 -0.00573284 -0.00573284 -0.00573284 -0.00573284 -0.00573284 -0.00573284]
[-0.52915962 -0.48486058 -0.52915962 -0.52915962 -0.52700963 -0.52915962
 -0.52915962 -0.52915962 -0.52915962 -0.52915962 -0.52915962 -0.52915962]
[-3.28938374 -3.2450847  -3.28938374 -3.28938374 -3.28723375 -3.28938374
 -3.28938374 -3.28938374 -3.28938374 -3.28938374 -3.28938374 -3.28938374]
[-0.00573284  0.0123649  -0.00537104 -0.00307311  0.01826251 -0.00573284
 -0.00437458 -0.00573284 -0.00573284 -0.00573284 -0.00573284 -0.00573284]
[-0.52915962 -0.51106188 -0.52879782 -0.52649989 -0.50516427 -0.52915962
 -0.52780136 -0.52915962 -0.52915962 -0.52915962 -0.52915962 -0.52915962]
[-3.28938374 -3.271286   -3.28902194 -3.28672401 -3.26538839 -3.28938374
 -3.28802548 -3.28938374 -3.28938374 -3.28938374 -3.28938374 -3.28938374]
[ 0.02266929  0.08268428 -0.00573284 -0.00573284 -0.00573284 -0.00573284
 -0.00573284 -0.00573284 -0.00573284 -0.00573

In [6]:
a = [-0.00573284, 0.0385662, -0.00573284, -0.00573284, -0.00358285, -0.00573284, -0.00573284, -0.00573284, -0.00573284, -0.00573284, -0.00573284, -0.00573284]
b = [-0.52915962, -0.48486058, -0.52915962, -0.52915962, -0.52700963, -0.52915962, -0.52915962, -0.52915962, -0.52915962, -0.52915962, -0.52915962, -0.52915962]
c = [-3.28938374, -3.2450847, -3.28938374, -3.28938374, -3.28723375, -3.28938374, -3.28938374, -3.28938374, -3.28938374, -3.28938374, -3.28938374, -3.28938374] 

print(list(np.asarray(a) + (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 11))
print(list(np.asarray(b) + (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 11))
print(list(np.asarray(c) +(-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 11))
print(sum(list(np.asarray(a) + (0.029371754418236085 + 0.0005659870105981069 * 39240) * 3.06889e-6 * 7 * 11)))
print(sum(list(np.asarray(b) + (7.193477398965072 + 0.05212817 * 39240) * 3.06889e-6 * 7 * 11)))
print(sum(list(np.asarray(c) +(-0.3309600825877492 + 0.3251892635925671 * 39240) * 3.06889e-6 * 7 * 11)))

[-0.00047773396419666884, 0.04382130603580334, -0.00047773396419666884, -0.00047773396419666884, 0.0016722560358033317, -0.00047773396419666884, -0.00047773396419666884, -0.00047773396419666884, -0.00047773396419666884, -0.00047773396419666884, -0.00047773396419666884, -0.00047773396419666884]
[-0.044096633500591675, 0.00020240649940839184, -0.044096633500591675, -0.044096633500591675, -0.04194664350059163, -0.044096633500591675, -0.044096633500591675, -0.044096633500591675, -0.044096633500591675, -0.044096633500591675, -0.044096633500591675, -0.044096633500591675]
[-0.2741153126269156, -0.22981627262691573, -0.2741153126269156, -0.2741153126269156, -0.27196532262691564, -0.2741153126269156, -0.2741153126269156, -0.2741153126269156, -0.2741153126269156, -0.2741153126269156, -0.2741153126269156, -0.2741153126269156]
0.04071622242963999
-0.4827105720071
-3.242934721522987
